#Upload images for training. The images should predominantly be close-ups. Include 2-3 full-body images for a broader perspective.



In [ ]:
cd /content/

#Now we install the lora github repo https://github.com/cloneofsimo/lora.git

In [ ]:
!pip install git+https://github.com/cloneofsimo/lora.git

We must resize the input images to a 512x512 resolution to facilitate expedited training. Assign the image location to the "instance_data_dir" parameter. The base model for training is determined by the "pretrained_model_name_or_path" parameter. The "lora_rank" parameter controls the image generation fidelity; a higher value results in higher fidelity but also extends training time. If the lora rank is increased, be sure to also increase the maximum training steps to prevent noisy generations.



In [ ]:
!lora_pti --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" --instance_data_dir="/content/input_images/" --output_dir="/content/output/" --train_text_encoder --resolution=512 \
--train_batch_size=1 --gradient_accumulation_steps=4 --scale_lr --learning_rate_unet=1e-4 --learning_rate_text=5e-5 --learning_rate_ti=5e-2 \
--color_jitter --lr_scheduler="linear" --lr_warmup_steps=0 --placeholder_tokens="<s1>" --initializer_tokens="man" --use_template="object" \
--save_steps=100 --max_train_steps_ti=1000 --max_train_steps_tuning=1000 --perform_inversion=True --clip_ti_decay --weight_decay_ti=0.000 \
--weight_decay_lora=0.001 --continue_inversion --continue_inversion_lr=1e-4 --device="cuda:0" --lora_rank=8

#After training, convert the resulting safetensors model diffusers model using the lora_add command and combining the safetensors with the base stable diffusion model

In [ ]:
!lora_add runwayml/stable-diffusion-v1-5 /content/output/final_lora.safetensors /content/output_merged 0.8 --mode upl

In [ ]:
!pip install diffusers==0.10.2 transformers scipy ftfy accelerate

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("/content/output_merged")
pipe.to("cuda")
prompt = "a photograph of a man <s1> riding a horse"

image = pipe(prompt).images[0]